In [1]:
import pyspark
import os

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = pyspark.SparkContext(appName="ex01", conf=conf)
sc.setLogLevel('OFF')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/27 08:53:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/10/27 08:53:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## 1.  Create an rdd using data from filtered_registered_business_sf.tsv

* Use 8 partitions.

In [2]:
# Load data from the file.
input_rdd = sc.textFile("../data/SF_business/supervisor_sf.tsv", 8)

In [3]:
input_rdd

../data/SF_business/supervisor_sf.tsv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

* rdd is data which is distributed.

## 2.  See the data in RDD

In [4]:
input_rdd.collect()

['94102\t8',
 '94102\t6',
 '94102\t3',
 '94102\t5',
 '94103\t8',
 '94103\t9',
 '94103\t10',
 '94103\t6',
 '94103\t3',
 '94103\t5',
 '94104\t6',
 '94104\t3',
 '94105\t6',
 '94105\t3',
 '94107\t10',
 '94107\t6',
 '94108\t6',
 '94108\t3',
 '94109\t2',
 '94109\t6',
 '94109\t3',
 '94109\t5',
 '94110\t8',
 '94110\t11',
 '94110\t9',
 '94110\t10',
 '94111\t6',
 '94111\t3',
 '94112\t7',
 '94112\t8',
 '94112\t11',
 '94112\t9',
 '94112\t10',
 '94114\t7',
 '94114\t8',
 '94114\t5',
 '94115\t2',
 '94115\t1',
 '94115\t5',
 '94116\t7',
 '94116\t4',
 '94117\t1',
 '94117\t7',
 '94117\t8',
 '94117\t5',
 '94118\t2',
 '94118\t1',
 '94118\t5',
 '94121\t2',
 '94121\t1',
 '94122\t1',
 '94122\t7',
 '94122\t5',
 '94122\t4',
 '94123\t2',
 '94124\t9',
 '94124\t10',
 '94127\t7',
 '94127\t8',
 '94127\t11',
 '94129\t2',
 '94130\t6',
 '94131\t7',
 '94131\t8',
 '94131\t5',
 '94132\t7',
 '94132\t11',
 '94132\t4',
 '94133\t2',
 '94133\t3',
 '94134\t11',
 '94134\t9',
 '94134\t10',
 '94158\t10',
 '94158\t6']

In [5]:
# It is no longer distributed! So do not use other than on the final step or for debugging.
type(input_rdd.collect())

list

## 3.  How many data are in RDD?

In [6]:
input_rdd.count()

75

In [7]:
# Also count() is an action like collect()
type(input_rdd.count())

int

## 4. See how data is distributed on 8 partitions

In [8]:
input_rdd.glom().collect()

[['94102\t8',
  '94102\t6',
  '94102\t3',
  '94102\t5',
  '94103\t8',
  '94103\t9',
  '94103\t10',
  '94103\t6',
  '94103\t3',
  '94103\t5'],
 ['94104\t6',
  '94104\t3',
  '94105\t6',
  '94105\t3',
  '94107\t10',
  '94107\t6',
  '94108\t6',
  '94108\t3',
  '94109\t2'],
 ['94109\t6',
  '94109\t3',
  '94109\t5',
  '94110\t8',
  '94110\t11',
  '94110\t9',
  '94110\t10',
  '94111\t6',
  '94111\t3'],
 ['94112\t7',
  '94112\t8',
  '94112\t11',
  '94112\t9',
  '94112\t10',
  '94114\t7',
  '94114\t8',
  '94114\t5',
  '94115\t2',
  '94115\t1'],
 ['94115\t5',
  '94116\t7',
  '94116\t4',
  '94117\t1',
  '94117\t7',
  '94117\t8',
  '94117\t5',
  '94118\t2',
  '94118\t1'],
 ['94118\t5',
  '94121\t2',
  '94121\t1',
  '94122\t1',
  '94122\t7',
  '94122\t5',
  '94122\t4',
  '94123\t2',
  '94124\t9',
  '94124\t10'],
 ['94127\t7',
  '94127\t8',
  '94127\t11',
  '94129\t2',
  '94130\t6',
  '94131\t7',
  '94131\t8',
  '94131\t5',
  '94132\t7'],
 ['94132\t11',
  '94132\t4',
  '94133\t2',
  '94133\t3',
  '9

## 5. Convert data in rdd to a tuple of integers

In [9]:
splitted_rdd = input_rdd.map(lambda x: x.split('\t'))

In [10]:
zip_id = splitted_rdd.map(lambda x: [int(x[0]), int(x[1])])

In [11]:
zip_id.collect()

[[94102, 8],
 [94102, 6],
 [94102, 3],
 [94102, 5],
 [94103, 8],
 [94103, 9],
 [94103, 10],
 [94103, 6],
 [94103, 3],
 [94103, 5],
 [94104, 6],
 [94104, 3],
 [94105, 6],
 [94105, 3],
 [94107, 10],
 [94107, 6],
 [94108, 6],
 [94108, 3],
 [94109, 2],
 [94109, 6],
 [94109, 3],
 [94109, 5],
 [94110, 8],
 [94110, 11],
 [94110, 9],
 [94110, 10],
 [94111, 6],
 [94111, 3],
 [94112, 7],
 [94112, 8],
 [94112, 11],
 [94112, 9],
 [94112, 10],
 [94114, 7],
 [94114, 8],
 [94114, 5],
 [94115, 2],
 [94115, 1],
 [94115, 5],
 [94116, 7],
 [94116, 4],
 [94117, 1],
 [94117, 7],
 [94117, 8],
 [94117, 5],
 [94118, 2],
 [94118, 1],
 [94118, 5],
 [94121, 2],
 [94121, 1],
 [94122, 1],
 [94122, 7],
 [94122, 5],
 [94122, 4],
 [94123, 2],
 [94124, 9],
 [94124, 10],
 [94127, 7],
 [94127, 8],
 [94127, 11],
 [94129, 2],
 [94130, 6],
 [94131, 7],
 [94131, 8],
 [94131, 5],
 [94132, 7],
 [94132, 11],
 [94132, 4],
 [94133, 2],
 [94133, 3],
 [94134, 11],
 [94134, 9],
 [94134, 10],
 [94158, 10],
 [94158, 6]]

* compare with flatmap() 

zip_id = splitted_rdd.flatMap(lambda x : [int(x[0]), int(x[1])])

## 6. Return unique zip code associated with id, 9 or 10

In [12]:
zip = zip_id.filter(lambda x: x[1] == 9 or x[1] == 10)\
            .map(lambda x: x[0])

In [13]:
distinct_zip = zip.distinct()

## 7. Order zip in ascending order

In [14]:
ordered_distinct_zip = distinct_zip.sortBy(lambda x: x, ascending=False)

In [15]:
ordered_distinct_zip.collect()

[94158, 94134, 94124, 94112, 94110, 94107, 94103]

In [16]:
sc.stop()